In [1]:
# repo rootz
%cd /work

/work


In [2]:
!ls 

LICENSE    docker    out    pyproject.toml  scripts  tests
README.md  examples  paper  resources	    src


In [3]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [4]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [5]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [3]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [4]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [5]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [11]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [17]:
# Fig2 run
#!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/*
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,context_swap,stress \
  --k-claims 50 \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
    --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,context_swap,stress
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 105
[run_fig2] (1/105) HNSC ours gate=hard tau=0.20
[run_fig2] (2/105) HNSC stress gate=hard tau=0.20
[run_fig2] (3/105) HNSC context_swap gate=hard tau=0.20
[run_fig2] (4/105) HNSC ours gate=hard tau=0.40
[run_fig2] (5/105) HNSC stress gate=hard tau=0.40
[run_fig2] (6/105) HNSC context_swap gate=hard tau=0.40
[run_fig2] (7/105) HNSC ours gate=hard tau=0.60
[run_fig2] (8/105) HNSC stress gate=hard tau=0.60
[run_fig2] (9/105) HNSC context_swap gate=hard tau=0.60
[run_fig2] (10/105) HNSC ours gate=hard tau=0.80
[run_fig2] (11/105) HNSC stress gate=hard tau=0.80
[run_

In [18]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root "/work/paper/source_data/PANCAN_TP53_v1/out" \
  --out "/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv"

[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv rows=105
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [20]:
import pandas as pd
df = pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv", sep="\t")
g = df.groupby(["condition","variant"])["tau"].nunique().sort_values()
print(g.head(20))
print("min n_tau =", g.min(), "max n_tau =", g.max(), "rows =", len(df))

condition  variant     
BRCA       context_swap    5
UCEC       context_swap    5
SKCM       stress          5
           ours            5
           context_swap    5
OV         stress          5
           ours            5
           context_swap    5
LUSC       stress          5
UCEC       ours            5
LUSC       ours            5
LUAD       stress          5
           ours            5
           context_swap    5
HNSC       stress          5
           ours            5
           context_swap    5
BRCA       stress          5
           ours            5
LUSC       context_swap    5
Name: tau, dtype: int64
min n_tau = 5 max n_tau = 5 rows = 105


In [81]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_abstain_variants.pdf \
  --ycol abstain_rate_total \
  --variants ours,context_swap,stress \
  --label-mode variant


In [75]:
!python paper/scripts/fig2_lines_status_by_tau.py \
  --in /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/FIg2_HNSC_abstain_vs_tau_variants.pdf \
  --condition HNSC \
  --gate-mode hard \
  --variants ours,context_swap,stress \
  --metric abstain \
  --label-mode proposed


In [76]:
!python paper/scripts/fig2_make_labels_template.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --condition HNSC \
  --per-run 30 \
  --prefer-status PASS \
  --with-hints \
  --rater-id R1 \
  --label-version v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv rows=49
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [55]:
!python paper/scripts/fig2_check_labels_merge.py \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv \
  --root   /work/paper/source_data/PANCAN_TP53_v1/out \
  --condition HNSC \
  --prefer-status PASS


[OK] labels claim_id unique
[OK] labels human_label values look valid (or blank)
[OK] audit logs used: 15
[OK] unique claim_id in audit (all statuses): 50
[OK] unique claim_id in audit (status=PASS): 43
[OK] labels rows: 49
[OK] labels claim_id found in audit: 49
[OK] non-blank labels total: 49
[OK] non-blank labels among audit status=PASS: 43
[INFO] fraction of non-blank labels that target status=PASS: 0.878
[DONE] If this is clean, pass --labels to fig2_collect_risk_coverage.py


In [58]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --method ours \
  --labels /work/paper/source_data/PANCAN_TP53_v1/labels/labels_hnsc_R1_v1.tsv


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv rows=105


In [84]:
!python paper/scripts/fig2_plot_scatter_human_risk.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/risk_coverage.label.tsv \
  --out /work/paper/source_data/PANCAN_TP53_v1/fig/Fig2_HNSC_scatter_human_risk.pdf \
  --condition HNSC \
  --ycol risk_human_nonaccept \
  --variants ours,context_swap,stress \
  --annotate-tau \
  --annotate-npass


In [69]:
import json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20/run_meta.json"
d=json.load(open(p))
print("status:", d.get("status"))
print("step:", d.get("step"))
print("error:", d.get("error"))
print("artifacts keys:", sorted((d.get("artifacts") or {}).keys()))
print("evidence_normalized_tsv:", (d.get("artifacts") or {}).get("evidence_normalized_tsv"))
print("inputs.evidence_normalized_sha256:", (((d.get("inputs") or {}).get("evidence_normalized_sha256"))))


status: ok
step: report_jsonl
error: None
artifacts keys: ['audit_log_tsv', 'claims_proposed_tsv', 'distilled_tsv', 'distilled_with_modules_tsv', 'evidence_normalized_tsv', 'modules_tsv', 'report_dir', 'report_jsonl', 'sample_card_normalized_json', 'sample_card_raw_json', 'term_gene_edges_tsv', 'term_modules_tsv']
evidence_normalized_tsv: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20/evidence.normalized.tsv
inputs.evidence_normalized_sha256: 9e65cad5e5526888b45dd65c72a50c63e5a71f5e35e406cea9b85ff79bc1c215


In [287]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC

context_swap  ours


In [288]:
import pandas as pd, json, glob
base=glob.glob("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20")[0]
p=base+"/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
print("context_review_mode unique:", sorted(set(df.get("context_review_mode", pd.Series(dtype=str)).astype(str))))
print("context_status counts:\n", df.get("context_status", pd.Series(dtype=str)).astype(str).value_counts(dropna=False).head(10))
print("first row context_notes:", df.get("context_notes", pd.Series(dtype=str)).astype(str).iloc[0] if len(df) else "")
m=json.load(open(base+"/run_meta.json"))
print("sample in meta:", m.get("inputs",{}).get("sample",{}))

context_review_mode unique: ['proxy']
context_status counts:
 context_status
PASS       40
ABSTAIN     8
FAIL        2
Name: count, dtype: int64
first row context_notes: proxy_context_v2: u=0.597 p_fail=0.050 p_warn=0.200 key=['condition', 'tissue', 'perturbation', 'comparison'] ctx_sig=e6cee1e46cac
sample in meta: {'condition': 'HNSC', 'tissue': 'tumor', 'perturbation': 'genotype', 'comparison': 'TP53_mut_vs_TP53_wt', 'species': ''}


In [289]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.normalized.json
claims.proposed.tsv	    report.jsonl	  sample_card.raw.json
distilled.tsv		    report.md		  term_gene_edges.tsv
distilled.with_modules.tsv  risk_coverage.tsv	  term_modules.tsv
evidence.normalized.tsv     run_meta.json
manifest.json		    run_meta.runner.json


In [290]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json

{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769479405_caf502b556",
  "status": "ok",
  "started_epoch": 1769479405.401813,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 50,
    "stress_evidence_dropout_p": 0.0,
    "stress_evidence_dropout_min_keep": 1,
    "stress_contradictory_p": 0.0,
    "stress_contradictory_max_extra": 0
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3",
    "tool_version": "0.0.0"
  },
  "inputs": {
    "evidence_table": {
      "path"

In [291]:
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"

import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")
print("shape", df.shape)
print(df["status"].value_counts(dropna=False))
for c in ["abstain_reason","fail_reason","context_status","context_evaluated","context_method","audit_notes"]:
    if c in df.columns:
        print("\n==", c, "==")
        print(df[c].fillna("").value_counts().head(20))

shape (50, 124)
status
PASS       34
ABSTAIN    16
Name: count, dtype: int64

== abstain_reason ==
abstain_reason
                       34
CONTEXT_GATE            7
unstable                7
context_nonspecific     2
Name: count, dtype: int64

== fail_reason ==
fail_reason
    50
Name: count, dtype: int64

== context_status ==
context_status
PASS       46
WARN        2
ABSTAIN     2
Name: count, dtype: int64

== context_evaluated ==
context_evaluated
True     43
False     7
Name: count, dtype: int64

== context_method ==
context_method
proxy    43
none      7
Name: count, dtype: int64

== audit_notes ==
audit_notes
                                                                                                                                                                  34
context_gate=hard                                                                                                                                                  7
survival[module]=0.766 < tau=0.80             

In [292]:
import json, pandas as pd

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")

def has_keys(s, keys):
    try:
        o = json.loads(str(s))
        return {k: (k in o) for k in keys}
    except Exception:
        return None

keys = ["context_evaluated","context_method","context_status","context_reason","context_confidence","context_notes"]
sample = df[df["abstain_reason"]=="context_missing"].head(5)

for i, r in sample.iterrows():
    ck = has_keys(r.get("claim_json",""), keys)
    print("row", i, "claim_json_has", ck)
    # ついでに中身の該当フィールドだけ見たいなら：
    try:
        o = json.loads(r["claim_json"])
        print({k:o.get(k,None) for k in keys})
    except Exception as e:
        print("json load error:", e)
    print("---")


In [293]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("\n".join(df.columns))

claim_id
entity
direction
context_keys
term_uid
source
term_id
term_name
module_id
module_missing
module_reason
gene_ids
gene_ids_suggest
gene_ids_snippet
gene_ids_n_total
gene_ids_n_in_claim
term_ids
gene_set_hash
context_score
context_evaluated
context_status
context_method
context_confidence
context_reason
select_context_score
select_context_evaluated
select_context_tokens_n
select_context_status_score
select_context_tiebreak
context_signature
context_swap_active
context_gate_mode
context_gate_hit
eligible_context
eligible
term_survival
keep_term
keep_reason
claim_json
preselect_tau_gate
context_score_proxy
select_context_mode
select_context_policy
select_notes
select_diag_n_total
select_diag_n_eligible
select_diag_n_ineligible
select_diag_n_picked
select_diag_blocked_by_module_cap
select_diag_relaxed
select_diag_relax_passes
select_diag_max_per_module
select_diag_ctx_hit_terms
claim_mode
llm_notes
context_notes
context_review_mode
context_ctx_id
context_proxy_method
context_score_p

In [294]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20
report   True 111910
audit    True 147950
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 476
meta     True 9244

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'PASS': 39, 'ABSTAIN': 11}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id method condition          comparison  tau    

In [295]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20
report   True 118654
audit    True 156599
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 611
meta     True 9685

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'ABSTAIN': 36, 'PASS': 14}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id       method condition          compa

In [296]:
import pandas as pd

base_ours = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20/audit_log.tsv"
base_swap = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20/audit_log.tsv"

a = pd.read_csv(base_ours, sep="\t")
b = pd.read_csv(base_swap, sep="\t")

def _ensure_decision(df: pd.DataFrame) -> pd.DataFrame:
    # Prefer explicit "decision"; otherwise fall back to "status" (common in audit logs)
    if "decision" not in df.columns and "status" in df.columns:
        df = df.copy()
        df["decision"] = df["status"]
    return df

a = _ensure_decision(a)
b = _ensure_decision(b)

want = [
    "claim_id",
    "term_id",
    "term_name",
    "decision",
    "abstain_reason",
    "context_score_proxy_u01",
    "context_reason",
    "context_swap_active",
    "context_ctx_id_original",
    "context_ctx_id_effective",
    "context_swap_to",
]

def _pick_existing(df: pd.DataFrame, cols: list[str]) -> list[str]:
    return [c for c in cols if c in df.columns]

keep_a = _pick_existing(a, want)
keep_b = _pick_existing(b, want)

aa = a[keep_a].rename(columns=lambda c: c if c == "claim_id" else f"{c}_ours")
bb = b[keep_b].rename(columns=lambda c: c if c == "claim_id" else f"{c}_swap")

m = aa.merge(bb, on="claim_id", how="inner")

# Find flips (ours vs swap)
d = m[(m.get("decision_ours") != m.get("decision_swap"))].copy()

print("cols_used_ours:", keep_a)
print("cols_used_swap:", keep_b)
print("n_flips:", len(d))

show = [c for c in [
    "claim_id",
    "term_name_ours", "term_name_swap",
    "decision_ours", "decision_swap",
    "abstain_reason_ours", "abstain_reason_swap",
    "context_score_proxy_u01_ours", "context_score_proxy_u01_swap",
    "context_swap_to_swap",
] if c in d.columns]

if len(d) == 0:
    print("No decision flips found between ours and context_swap at this tau.")
else:
    print(d[show].to_string(index=False))


cols_used_ours: ['claim_id', 'term_id', 'term_name', 'decision', 'abstain_reason', 'context_score_proxy_u01', 'context_reason', 'context_swap_active', 'context_ctx_id_original', 'context_ctx_id_effective', 'context_swap_to']
cols_used_swap: ['claim_id', 'term_id', 'term_name', 'decision', 'abstain_reason', 'context_score_proxy_u01', 'context_reason', 'context_swap_active', 'context_ctx_id_original', 'context_ctx_id_effective', 'context_swap_to']
n_flips: 33
      claim_id                           term_name_ours                           term_name_swap decision_ours decision_swap abstain_reason_ours abstain_reason_swap  context_score_proxy_u01_ours  context_score_proxy_u01_swap context_swap_to_swap
c_a109d5f2a3c3                       HALLMARK_APOPTOSIS                       HALLMARK_APOPTOSIS          PASS       ABSTAIN                 NaN        CONTEXT_GATE                      0.822900                      0.918818                 LUAD
c_3de6c9d2aa73       HALLMARK_OXIDATIVE_PHOSPH

In [297]:
import pandas as pd

def probe_u_cols(df):
    cand = [c for c in df.columns if ("proxy" in c and "u01" in c) or c.endswith("_u01") or "u_gate" in c]
    out = []
    for c in cand:
        s = pd.to_numeric(df[c], errors="coerce")
        out.append((c, float(s.min()), float(s.max()), int(s.notna().sum())))
    return pd.DataFrame(out, columns=["col","min","max","n_non_na"]).sort_values(["n_non_na","col"], ascending=[False,True])

print("OURS")
display(probe_u_cols(a))   # a は ours の audit df
print("SWAP")
display(probe_u_cols(b))   # b は swap の audit df


OURS


,col,min,max,n_non_na
1,context_score_proxy_u01,0.004311,0.940425,50
3,context_score_proxy_u01_from_proposed,0.025939,0.999977,50
2,context_score_proxy_u01_norm,0.025939,0.999977,50
4,context_score_proxy_u01_norm_from_proposed,0.025939,0.999977,50
0,preselect_tau_gate,0.000000,0.000000,50


SWAP


,col,min,max,n_non_na
1,context_score_proxy_u01,0.005629,0.965492,50
3,context_score_proxy_u01_from_proposed,0.026690,0.981552,50
2,context_score_proxy_u01_norm,0.026690,0.981552,50
4,context_score_proxy_u01_norm_from_proposed,0.026690,0.981552,50
0,preselect_tau_gate,0.000000,0.000000,50


In [298]:
p_warn = 0.2

keep = [
    "claim_id","term_name","decision","abstain_reason",
    "context_score_proxy_u01","context_score_proxy_u01_norm",
    "context_ctx_id_effective","context_ctx_id_original",
    "context_swap_to","context_swap_active",
    "context_gate_hit","eligible","status",  # ここが超重要（あれば）
]

aa = a[[c for c in keep if c in a.columns]].rename(columns=lambda c: c+"_ours" if c!="claim_id" else c)
bb = b[[c for c in keep if c in b.columns]].rename(columns=lambda c: c+"_swap" if c!="claim_id" else c)
m = aa.merge(bb, on="claim_id", how="inner")

m["u_ours"] = pd.to_numeric(m.get("context_score_proxy_u01_ours"), errors="coerce")
m["u_swap"] = pd.to_numeric(m.get("context_score_proxy_u01_swap"), errors="coerce")
m["hit_ours"] = m["u_ours"] < p_warn
m["hit_swap"] = m["u_swap"] < p_warn

flips = m[m["decision_ours"] != m["decision_swap"]].copy()
flips["crossed_threshold"] = flips["hit_ours"] != flips["hit_swap"]
flips["delta_u"] = flips["u_swap"] - flips["u_ours"]

show_cols = [
    "claim_id",
    "term_name_ours","term_name_swap",
    "decision_ours","decision_swap",
    "abstain_reason_ours","abstain_reason_swap",
    "u_ours","u_swap","delta_u",
    "hit_ours","hit_swap","crossed_threshold",
    "context_gate_hit_ours","context_gate_hit_swap",
    "eligible_ours","eligible_swap",
    "status_ours","status_swap",
    "context_ctx_id_effective_ours","context_ctx_id_effective_swap",
    "context_swap_to_swap",
]
show_cols = [c for c in show_cols if c in flips.columns]

flips_sorted = flips.sort_values(["crossed_threshold","delta_u"], ascending=[False, True])
display(flips_sorted[show_cols])
print("n_flips:", len(flips_sorted))
print("n_crossed_threshold:", int(flips_sorted["crossed_threshold"].sum()))


,claim_id,term_name_ours,term_name_swap,decision_ours,decision_swap,abstain_reason_ours,abstain_reason_swap,u_ours,u_swap,delta_u,...,crossed_threshold,context_gate_hit_ours,context_gate_hit_swap,eligible_ours,eligible_swap,status_ours,status_swap,context_ctx_id_effective_ours,context_ctx_id_effective_swap,context_swap_to_swap
12,c_7eb60753a906,HALLMARK_KRAS_SIGNALING_UP,HALLMARK_KRAS_SIGNALING_UP,PASS,ABSTAIN,NaN,context_nonspecific,0.790465,0.028343,-0.762122,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
42,c_3906e8d8f560,HALLMARK_G2M_CHECKPOINT,HALLMARK_G2M_CHECKPOINT,PASS,ABSTAIN,NaN,context_nonspecific,0.739028,0.016527,-0.722501,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
47,c_301093d58998,HALLMARK_COAGULATION,HALLMARK_COAGULATION,PASS,ABSTAIN,NaN,context_nonspecific,0.681804,0.005629,-0.676175,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
4,c_02aa46f2c7ef,HALLMARK_COMPLEMENT,HALLMARK_COMPLEMENT,PASS,ABSTAIN,NaN,context_nonspecific,0.700764,0.097641,-0.603123,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
45,c_0a5a95d3c933,HALLMARK_ADIPOGENESIS,HALLMARK_ADIPOGENESIS,PASS,ABSTAIN,NaN,context_nonspecific,0.553841,0.168041,-0.385800,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
20,c_5db50990e92d,HALLMARK_ESTROGEN_RESPONSE_EARLY,HALLMARK_ESTROGEN_RESPONSE_EARLY,PASS,ABSTAIN,NaN,context_nonspecific,0.426932,0.098673,-0.328259,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
6,c_40f168bf3a97,HALLMARK_FATTY_ACID_METABOLISM,HALLMARK_FATTY_ACID_METABOLISM,PASS,ABSTAIN,NaN,context_nonspecific,0.162716,0.205817,0.043101,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
13,c_ba0bf933a576,HALLMARK_ANDROGEN_RESPONSE,HALLMARK_ANDROGEN_RESPONSE,PASS,ABSTAIN,NaN,context_nonspecific,0.152763,0.225522,0.072759,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
36,c_79cd2596fe00,HALLMARK_UNFOLDED_PROTEIN_RESPONSE,HALLMARK_UNFOLDED_PROTEIN_RESPONSE,PASS,ABSTAIN,NaN,context_nonspecific,0.107902,0.207293,0.099391,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
44,c_87844015517c,HALLMARK_GLYCOLYSIS,HALLMARK_GLYCOLYSIS,PASS,ABSTAIN,NaN,context_nonspecific,0.157992,0.333330,0.175338,...,True,False,True,True,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD


n_flips: 33
n_crossed_threshold: 10


In [299]:
# swap のみでOK
b[["context_gate_hit","eligible_context","context_status","select_context_status_score","context_score_proxy_u01"]].head(10)
print(b["context_status"].value_counts(dropna=False).head(10))
print(b["select_context_status_score"].value_counts(dropna=False).head(10))
print(b["context_gate_hit"].value_counts(dropna=False))
print(b["eligible_context"].value_counts(dropna=False))


context_status
WARN    32
PASS    18
Name: count, dtype: int64
select_context_status_score
0    32
1    18
Name: count, dtype: int64
context_gate_hit
True     32
False    18
Name: count, dtype: int64
eligible_context
False    32
True     18
Name: count, dtype: int64


In [300]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.normalized.json
claims.proposed.tsv	    report.jsonl	  sample_card.raw.json
distilled.tsv		    report.md		  term_gene_edges.tsv
distilled.with_modules.tsv  risk_coverage.tsv	  term_modules.tsv
evidence.normalized.tsv     run_meta.json
manifest.json		    run_meta.runner.json


In [301]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 112179
audit    True 147329
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 476
meta     True 9242

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'PASS': 34, 'ABSTAIN': 16}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id method condition          comparison  tau    

In [302]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80
report   True 117839
audit    True 154765
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 611
meta     True 9686

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'ABSTAIN': 37, 'PASS': 13}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id       method condition          compa

In [303]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants stress \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 100 \
  --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: stress
  gate_modes: hard
  taus: 0.80
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC stress gate=hard tau=0.80
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [304]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 112179
audit    True 147329
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 476
meta     True 9242

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'PASS': 34, 'ABSTAIN': 16}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id method condition          comparison  tau    

In [307]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
report   False None
audit    False None
modules  False None
edges    False None
terms    False None
distilled False None
risk     False None
card     False None
meta     False None
[SANITY][ERROR] missing required artifact: report -> /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80/report.jsonl


In [308]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*

!LLMPATH_BACKEND=ollama \
LLMPATH_OLLAMA_HOST=http://ollama:11434 \
LLMPATH_OLLAMA_MODEL=llama3.1:8b \
LLMPATH_CONTEXT_REVIEW_MODE=llm \
LLMPATH_CONTEXT_GATE_MODE=hard \
LLMPATH_DEBUG=1 \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 50 \
  --context-review-mode llm \
  --force

import json
m=json.load(open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json"))
print(m["inputs"]["llm"])

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df = pd.read_csv(p, sep="\t")
print(df["context_method"].fillna("").value_counts().head(10))
print(df["context_evaluated"].fillna(False).astype(bool).value_counts())

!cp "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv" "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log_LLM.tsv"

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
[select] mode_eff=deterministic backend=yes review_backend=yes k=50 ctx_review_mode=llm
[context_review] updated=50 shortlist=50 missing=50
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
{'claim': {'enabled': False, 'claim_mode_env': '', 'backend_env': 'ollama', 'backend_enabled': False, 'backend_notes': ''}, 'review': {'enabled': True, 'review_mode': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}, 'select_entrypoint': {'backend_attached': True, 'backend_role': 'review'}}
context_method
none    50
Name: count, dtype: int64
context_evaluated
False    50
Name: count, dtype: int64


In [309]:
import pandas as pd
df=pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv", sep="\t")
print(df["status"].value_counts(dropna=False))

print(df["context_method"].fillna("").value_counts().head(10))
print(df["context_evaluated"].fillna(False).astype(bool).value_counts())



status
ABSTAIN    26
PASS       24
Name: count, dtype: int64
context_method
proxy    43
none      7
Name: count, dtype: int64
context_evaluated
True     43
False     7
Name: count, dtype: int64


In [311]:
import json
from collections import Counter
c=Counter()
with open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl","r",encoding="utf-8") as f:
    for line in f:
        o=json.loads(line)
        c[o.get("decision_status")] += 1
print(c)

Counter({'ABSTAIN': 26, 'PASS': 24})


In [312]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log_LLM.tsv"
df=pd.read_csv(p, sep="\t")

print("review_mode:", df["context_review_mode"].value_counts(dropna=False).to_dict())
print("tokens_sum:", df["select_context_tokens_n"].fillna(0).astype(int).sum())
print("tokens_dist:", df["select_context_tokens_n"].fillna(0).astype(int).value_counts().head(10).to_dict())

x = df[df["abstain_reason"].fillna("").eq("CONTEXT_GATE")].copy()

cols = [c for c in [
  "claim_id","decision","abstain_reason",
  "context_gate_mode","context_gate_blocked","context_gate_hit",
  "eligible_context","eligible",
  "context_method","context_evaluated","context_status","context_reason","context_confidence",
  "select_context_mode","select_context_status_score","select_context_score","select_context_tokens_n",
  "context_method_from_proposed","context_status_from_proposed","context_reason_from_proposed","context_confidence_from_proposed"
] if c in x.columns]

print(x[cols].head(30).to_string(index=False))
print("\ncontext_method:", x["context_method"].value_counts(dropna=False).to_dict())
print("context_method_from_proposed:", x["context_method_from_proposed"].value_counts(dropna=False).to_dict())

y = df[df["abstain_reason"].fillna("").eq("CONTEXT_GATE")].copy()
cols = [c for c in ["claim_id","decision","abstain_reason",
                    "context_status_from_proposed","context_reason_from_proposed","context_confidence_from_proposed",
                    "select_context_status_score","select_context_tokens_n"] if c in y.columns]
print(y[cols].head(25).to_string(index=False))


p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df = pd.read_csv(p, sep="\t")
print(df["context_method"].fillna("").value_counts().head(10))
print(df["context_status"].fillna("").value_counts().head(10))
print(df["context_evaluated"].fillna(False).astype(bool).value_counts())

review_mode: {'llm': 50}
tokens_sum: 0
tokens_dist: {0: 50}
      claim_id abstain_reason context_gate_mode  context_gate_blocked  context_gate_hit  eligible_context  eligible context_method  context_evaluated context_status      context_reason  context_confidence select_context_mode  select_context_status_score  select_context_score  select_context_tokens_n context_method_from_proposed context_status_from_proposed                                                                                                                                                                                                                                                            context_reason_from_proposed  context_confidence_from_proposed
c_12ba6a29e800   CONTEXT_GATE              hard                 False             False              True     False          proxy               True           PASS proxy_context_score                 0.0                 off                            1              

In [313]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 112426
audit    True 151279
modules  True 15385
edges    True 88967
terms    True 3188
distilled True 42580
risk     True 1615
card     True 476
meta     True 8886

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 68)
[report] decisions: {'ABSTAIN': 26, 'PASS': 24}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
  benchmark_id                run_id method condition          comparison  tau    

In [314]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*

!LLMPATH_BACKEND="ollama" \
LLMPATH_OLLAMA_HOST="http://ollama:11434" \
LLMPATH_OLLAMA_MODEL="llama3.1:8b" \
LLMPATH_CONTEXT_REVIEW_MODE=llm \
LLMPATH_CONTEXT_GATE_MODE=hard \
LLMPATH_CLAIM_MODE=llm \
LLMPATH_DEBUG=1 \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 3 \
#  --context-review-mode llm \
  --force

OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!cat $OUT/llm_claims.raw.json
!cat $OUT/llm_claims.meta.json

import json
m=json.load(open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json"))
print(m["inputs"]["llm"])

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 3
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
Traceback (most recent call last):
  File "/work/paper/scripts/run_fig2_pipeline.py", line 1054, in <module>
    main()
  File "/work/paper/scripts/run_fig2_pipeline.py", line 1047, in main
    _run_one(job, force=bool(args.force), plan=plan)
  File "/work/paper/scripts/run_fig2_pipeline.py", line 605, in _run_one
    run_pipeline(cfg)
  File "/opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py", line 2852, in run_pipeline
    _safe_mkdir_outdir(cfg.outdir, cfg.force)
  File "/opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py", line 100, in _safe_mkdir_outdir
    raise FileExistsError(f"outdir is not empty: {outdir} (use --force)")
FileExistsError: outdir is not empty: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/

FileNotFoundError: [Errno 2] No such file or directory: '/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json'

In [ ]:
import pandas as pd
df=pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv", sep="\t")
print([c for c in df.columns if c.startswith("context")])

In [ ]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

!ls -lh $OUT/{run_meta.json,claims.proposed.tsv,audit_log.tsv,report.jsonl} 2>/dev/null || true
!wc -l  $OUT/claims.proposed.tsv  $OUT/audit_log.tsv  $OUT/report.jsonl 2>/dev/null || true

import json, os
p=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")+"/run_meta.json"
m=json.load(open(p))
print("step:", m.get("step"), "status:", m.get("status"))
print("llm:", m.get("inputs",{}).get("llm",{}))
print("claims:", m.get("inputs",{}).get("claims",{}))

In [ ]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

In [159]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
p=f"{out}/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
cols=[c for c in df.columns if c.startswith("context_review_")]
print("context_review_* cols:", cols)
print(df[cols].head(3) if cols else "(none)")

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
df=pd.read_csv(f"{out}/claims.proposed.tsv", sep="\t")
cj=df.loc[0,"claim_json"]
obj=json.loads(cj)
keys=[k for k in obj.keys() if k.startswith("context_")]
print("claim_json context_* keys:", keys)
print({k: obj.get(k) for k in keys})

context_review_* cols: ['context_review_mode']
  context_review_mode
0               proxy
1               proxy
2               proxy
claim_json context_* keys: ['context_keys', 'context_evaluated', 'context_method', 'context_status', 'context_reason', 'context_notes']
{'context_keys': ['condition', 'tissue', 'perturbation', 'comparison'], 'context_evaluated': False, 'context_method': 'none', 'context_status': None, 'context_reason': None, 'context_notes': None}


In [160]:
import json, pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
row=df.iloc[0]
cj=json.loads(row["claim_json"])
print("context_keys:", cj.get("context_keys"))

context_keys: ['condition', 'tissue', 'perturbation', 'comparison']


In [161]:
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("cols:", [c for c in df.columns if "claim_json" in c])
print("has_claim_json:", "claim_json" in df.columns)
print("has_claim_json_str:", "claim_json_str" in df.columns)

cols: ['claim_json']
has_claim_json: True
has_claim_json_str: False


In [162]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!ls -lh "$OUT" | sed -n '1,200p'

for f in ["distilled.tsv", "distilled_with_modules.tsv modules.tsv", "evidence.normalized.tsv", "evidence.tsv",]:
  ![ -f "$OUT/$f" ] && echo "== $f ==" && wc -l "$OUT/$f" && head -n 3 "$OUT/$f"



total 460K
-rw-r--r-- 1 root root  14K Jan 26 15:25 audit_log.tsv
-rw-r--r-- 1 root root 7.6K Jan 26 15:25 claims.proposed.tsv
-rw-r--r-- 1 root root  41K Jan 26 15:25 distilled.tsv
-rw-r--r-- 1 root root  41K Jan 26 15:20 distilled.with_modules.tsv
-rw-r--r-- 1 root root  16K Jan 26 15:20 evidence.normalized.tsv
-rw-r--r-- 1 root root 124K Jan 26 15:25 llm_claims.candidates.json
-rw-r--r-- 1 root root  643 Jan 26 15:25 llm_claims.meta.json
-rw-r--r-- 1 root root 7.4K Jan 26 15:25 llm_claims.prompt.json
-rw-r--r-- 1 root root  684 Jan 26 15:25 llm_claims.raw.json
-rw-r--r-- 1 root root 2.3K Jan 26 15:25 manifest.json
-rw-r--r-- 1 root root  14K Jan 26 15:20 modules.tsv
-rw-r--r-- 1 root root 7.0K Jan 26 15:25 report.jsonl
-rw-r--r-- 1 root root  44K Jan 26 15:25 report.md
-rw-r--r-- 1 root root  398 Jan 26 15:25 risk_coverage.tsv
-rw-r--r-- 1 root root 8.4K Jan 26 15:25 run_meta.json
-rw-r--r-- 1 root root  641 Jan 26 15:20 run_meta.runner.json
-rw-r--r-- 1 root root  476 Jan 26 15:20 

In [163]:
import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/distilled.tsv"
df = pd.read_csv(p, sep="\t")
print(df.shape)
for c in ["keep_term","eligible","term_survival","keep_reason"]:
    if c in df.columns:
        print(c, df[c].value_counts(dropna=False).head(10))

(50, 56)
keep_term keep_term
True    50
Name: count, dtype: int64
term_survival term_survival
0.984375    10
1.000000     7
0.968750     7
0.953125     5
0.921875     4
0.906250     3
0.937500     3
0.750000     2
0.843750     2
0.875000     1
Name: count, dtype: int64
keep_reason keep_reason
ok    50
Name: count, dtype: int64


In [164]:
import pandas as pd, json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
print("rows:", df.shape[0])
print("context_confidence unique:", df.get("context_confidence").unique() if "context_confidence" in df.columns else None)
print("context_review_confidence unique:", df.get("context_review_confidence").unique() if "context_review_confidence" in df.columns else None)


rows: 3
context_confidence unique: [0.59721803 0.30170462 0.93198145]
context_review_confidence unique: None


In [165]:
# context未評価でABSTAINになっているか
!grep -n '"decision_status": "ABSTAIN"' \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head

# audit_notes に context_missing(hard) が残っているか
!grep -n "context_missing(hard)" \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head


In [166]:
# 1) DNS/ネットワーク
!getent hosts ollama || ping -c 1 ollama

# 2) Ollama API 生存確認
!curl -sS http://ollama:11434/api/tags | head

# 3) 推論が通るか（最小）
!curl -sS http://ollama:11434/api/generate \
  -H 'Content-Type: application/json' \
  -d '{"model":"llama3.1:8b","prompt":"Say OK","stream":false}' | head


172.18.0.2      ollama
{"models":[{"name":"llama3.1:8b","model":"llama3.1:8b","modified_at":"2026-01-01T16:13:01.415171001Z","size":4920753328,"digest":"46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_K_M"}}]}{"model":"llama3.1:8b","created_at":"2026-01-26T15:25:53.519164881Z","response":"OK","done":true,"done_reason":"stop","context":[128006,882,128007,271,46864,10619,128009,128006,78191,128007,271,4012],"total_duration":918740542,"load_duration":152680958,"prompt_eval_count":12,"prompt_eval_duration":663689042,"eval_count":2,"eval_duration":97652958}

In [856]:
import os, json, requests
host = os.environ.get("LLMPATH_OLLAMA_HOST","http://ollama:11434")
model = os.environ.get("LLMPATH_OLLAMA_MODEL","llama3.1:8b")
payload = {"model": model, "prompt": "Say OK", "stream": False}
r = requests.post(f"{host}/api/generate", json=payload, timeout=120)
print("status", r.status_code)
print("text", r.text[:200])
print("json", r.json())

status 200
text {"model":"llama3.1:8b","created_at":"2026-01-25T21:05:31.855906428Z","response":"OK! What's next?","done":true,"done_reason":"stop","context":[128006,882,128007,271,46864,10619,128009,128006,78191,128
json {'model': 'llama3.1:8b', 'created_at': '2026-01-25T21:05:31.855906428Z', 'response': "OK! What's next?", 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 46864, 10619, 128009, 128006, 78191, 128007, 271, 4012, 0, 3639, 596, 1828, 30], 'total_duration': 774175792, 'load_duration': 110849875, 'prompt_eval_count': 12, 'prompt_eval_duration': 114504875, 'eval_count': 7, 'eval_duration': 543352584}


In [ ]:
import inspect
import llm_pathway_curator.backends as b
print("backends.py:", b.__file__)
print("--- OllamaBackend.generate source ---")
print(inspect.getsource(b.OllamaBackend.generate))

In [ ]:
from llm_pathway_curator.backends import get_backend_from_env
b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, getattr(b, "host", None), getattr(b, "model_name", None))

s = b.generate('{"ping": "pong"}', json_mode=True)
print("returned:", s[:200])

In [ ]:
!grep -RIn "backend_call_failed" -n src/llm_pathway_curator
!grep -RIn "llm_claims.raw.json" -n src/llm_pathway_curator
!grep -RIn "llm_claims.meta.json" -n src/llm_pathway_curator


In [ ]:
import json
from llm_pathway_curator.backends import get_backend_from_env

b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, b.host, b.model_name)

# 1) json_mode で “スキーマっぽい” JSONを返させる
prompt = '{"claims": [{"claim_id":"c1","entity":"X","direction":"up","evidence_refs":["m1"],"context":{"disease":"HNSC"} }]}'
out = b.generate(prompt, json_mode=True)
print("raw_out_head:", out[:200])

# 2) ちゃんとJSONとして読めるか
obj = json.loads(out)
print("top_keys:", list(obj.keys())[:10])

In [77]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [142]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 70
[run_fig2] (1/70) HNSC ours gate=hard tau=0.20
[run_fig2] (2/70) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/70) HNSC ours gate=hard tau=0.40
[run_fig2] (4/70) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/70) HNSC ours gate=hard tau=0.60
[run_fig2] (6/70) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/70) HNSC ours gate=hard tau=0.80
[run_fig2] (8/70) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/70) HNSC ours gate=hard tau=0.90
[run_fig2] (10/70) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] (11/70) LUAD ours gate=hard tau=0.20
[run_fig2] (12/70) LUAD shuffled_context gate=hard tau=0.20
[run_fig2] (13/70) LUAD ours gate=hard tau=0.40
[run_fig2] (14/70) LUAD shuffled_c

In [143]:
!python paper/scripts/fig2_make_labels_template.py \
  --root paper/source_data/PANCAN_TP53_v1/out \
  --out  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --per-run 20 \
  --prefer-status PASS \
  --with-hints

[OK] wrote: paper/source_data/PANCAN_TP53_v1/labels.tsv rows=150
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [141]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [144]:
!python paper/scripts/bench_fig2.py \
  --reports paper/source_data/PANCAN_TP53_v1/out \
  --labels  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --out     /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --taus    0:1:0.05 \
  --rater-id R1


Traceback (most recent call last):
  File "/work/paper/scripts/bench_fig2.py", line 315, in <module>
    main()
  File "/work/paper/scripts/bench_fig2.py", line 303, in main
    reports_df = load_reports(report_paths)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/paper/scripts/bench_fig2.py", line 123, in load_reports
    raise ValueError(
ValueError: Missing metrics.survival in paper/source_data/PANCAN_TP53_v1/out/BRCA/ours/gate_hard/tau_0.20/report.jsonl line 1 (claim_id=c_a1ed09ef8099)


In [315]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_proxy.pdf \
  --ycol risk_proxy \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
